In [27]:
import os
import time
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess_input
from tensorflow.keras.models import load_model

# Class labels for EfficientNet and DenseNet
eff_class_labels = {0:'Healthy Fruit', 1:'Healthy Leaf', 2: 'Mealy Bug', 3: 'Mite Bug'}  # EfficientNet labels
dense_class_labels = {0: 'Mealy Bug', 1: 'Mite Bug'}  # DenseNet labels

# Load EfficientNet model
eff_model_path = "efficientNet_model/efficient_classify_model.h5"
eff_model = load_model(eff_model_path)

# Load DenseNet model
dense_model_path = "saved_modal/papaya_disease_model.h5"
dense_model = load_model(dense_model_path)

In [33]:
# Define class labels
eff_class_labels = {0: "fruit", 1: "leaf", 2: "other"}  # EfficientNet classes
dense_class_labels = {0:'Healthy Fruit', 1:'Healthy Leaf', 2: 'Mealy Bug', 3: 'Mite Bug'}   # DenseNet classes

eff_threshold = 0.8   # Threshold for EfficientNet

def preprocess_image(img_path, target_size, model_type='efficientnet'):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)

    if model_type == 'efficientnet':
        img_array = eff_preprocess_input(img_array)  # EfficientNet preprocessing
    elif model_type == 'densenet':
        img_array = img_array / 255.0  # Normalize for DenseNet

    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

def predict_eff(img_path):
    processed_img = preprocess_image(img_path, target_size=(224, 224), model_type='efficientnet')
    
    # Measure inference time
    start_time = time.time()
    predictions = eff_model.predict(processed_img)
    inference_time = time.time() - start_time

    eff_index = np.argmax(predictions[0])
    eff_confidence = predictions[0][eff_index]

    if eff_confidence < eff_threshold:
        return "Unrecognized", eff_confidence, None, inference_time

    eff_label = eff_class_labels[eff_index]
    return eff_label, eff_confidence, processed_img, inference_time

def predict_dense(img_array):
    # Measure inference time
    start_time = time.time()
    predictions = dense_model.predict(img_array)
    inference_time = time.time() - start_time

    dense_index = np.argmax(predictions[0])
    dense_confidence = predictions[0][dense_index]

    dense_label = dense_class_labels[dense_index]
    return dense_label, dense_confidence, inference_time

# Path to the test image
image_path = os.path.join('densenet_dataset', 'splitted_data', 'test', 'Mealy Bug', '20240713_151830.jpg')

# To verify the path
print("Image Path:", image_path)

# Predict with EfficientNet whether fruit or leaf
eff_label, eff_confidence, processed_img, eff_inference_time = predict_eff(image_path)
print(f"EfficientNet Prediction: {eff_label}, Confidence: {eff_confidence:.2f}, Inference Time: {eff_inference_time:.4f} seconds")

# If classified as "fruit" or "leaf," pass to DenseNet
if eff_label in ["fruit", "leaf"] and processed_img is not None:
    # Re-preprocess for DenseNet
    dense_img = preprocess_image(image_path, target_size=(224, 224), model_type='densenet')
    dense_label, dense_confidence, dense_inference_time = predict_dense(dense_img)
    print(f"DenseNet Prediction: {dense_label}, Confidence: {dense_confidence:.2f}, Inference Time: {dense_inference_time:.4f} seconds")
else:
    print("Image is not a 'fruit' or 'leaf'")


Image Path: densenet_dataset\splitted_data\test\Mealy Bug\20240713_151830.jpg
1/1 [==============================] - 0s 31ms/step
EfficientNet Prediction: fruit, Confidence: 1.00, Inference Time: 0.0640 seconds
1/1 [==============================] - 0s 49ms/step
DenseNet Prediction: Mealy Bug, Confidence: 0.99, Inference Time: 0.1290 seconds
